# example code for checking installation

In [1]:
import pyomo.environ as pyo
from pyomo2h5 import save_h5, save_dict_w_metadata

import h5py

In [2]:
# Dict to track constraints with unique keys
constraint_log = {}
constraint_counter = 0

def add_tracked_constraint(model, constraint_expr, name=None):
    global constraint_counter
    cname = name or f"added_constraint_{constraint_counter}"
    model.add_component(cname, pyo.Constraint(expr=constraint_expr))
    constraint_log[cname] = constraint_expr
    constraint_counter += 1


In [4]:


# Create Model
m = pyo.ConcreteModel("name")

m.Scenarios = pyo.RangeSet(3)

m.I = pyo.RangeSet(5)

m.p = pyo.Param(initialize=5,doc="this is a parameter")

m.x = pyo.Var(m.I, within=pyo.NonNegativeReals,doc="this is an indexed variable")

@m.Objective(doc="This is an Objective", sense=pyo.minimize)
def obj(m):
    return sum(m.x[i] for i in m.I)


@m.Block()
def scenario(block):
    block.Scenarios = pyo.RangeSet(5)
    block.x = pyo.Var(block.Scenarios, within=pyo.NonNegativeIntegers)
    @block.Constraint()
    def block_con1(block):
        return sum(block.x[j] for j in block.Scenarios) >= 34

@m.Constraint(m.I, doc="This is a Constraint")
def con1(m, i):
    return m.x[i] >= m.scenario.x[i]/10


add_tracked_constraint(m, m.x[2]<=5, "limit_x2_to_5")

del m.limit_x2_to_5

add_tracked_constraint(m, m.x[2]<=3, "limit_x2_to_5")


# Solve Model
filename = "test"

solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["LogFile"] = filename + ".log"
solver.options["TimeLimit"] = 60
results = solver.solve(m, tee=True)

# Save the dictionary to an HDF5 file
save_h5(m, results, filename,
        solver_options=solver.options,
        save_log_flag=False,
        save_constraint_flag=True,
        pickle_flag=True,
        git_flag=True)
print("Dictionary saved to HDF5 file successfully!")


def save_tracked_constraints_to_hdf5(file_path, constraint_dict):
    with h5py.File(file_path +".h5", 'a') as f:
        constraints_grp = f.create_group("constraints_newly_added")
        for name, expr in constraint_dict.items():
            constraints_grp.create_dataset(name, data=str(expr))




save_tracked_constraints_to_hdf5(filename, constraint_log)


data_dict2 = {"Metadata": {"Infos": "This will be metadata to the toplevel of the h5 file"}, "comment": {"Content": "This comes as data to the comment group", "Metadata": {"Info": "This will be the attribute Info and its text", "Written by": "Julius", "For": "Julius"}}, "Variable": {"Metadata": {"Info": "This is metadata to the variables"}}}

save_dict_w_metadata(filename, data_dict2)

Set parameter OutputFlag to value 1
Set parameter LogFile to value "test.log"
Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD Ryzen 7 PRO 5850U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 7 rows, 10 columns and 16 nonzeros
Model fingerprint: 0xb9b9f45f
Variable types: 5 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+01]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3.4 

Optimal solution found (tolerance 1.00e-04)
Best obje